In [1]:
from Tool_Functions.cleaning_data import *
from Tool_Functions.comportment_reabo import * 
from Tool_Functions.join_data import * 

In [2]:
path_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"
path_results_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Results/"

Nouveaux indicateurs : 
- Mois du réabonnement 
- Delai de réabonnement moyen pour cette promo comparé au délai de réabonnement moyen pour toutes les promos
- Trouver aussi un moyen de quantifier le poids de l’abonnement par rapport aux habitudes de fidélité d’un client : s’il s’abonne alors qu’il s’abonne jamais, la promo aura un poids plus important

In [3]:
df = file_to_dataframe(path_antoine + "df_Donnees_Reabos.csv")

In [10]:
dfp = file_to_dataframe(path_antoine + "df_Donnees_Promos.csv")

In [4]:
odd = file_to_dataframe(path_antoine + "df_Donnees_Reabos_odd.csv")

In [9]:
oddp = file_to_dataframe(path_antoine + "df_Donnees_Promos_odd.csv")

In [17]:
oddp.shape

(11415480, 5)

In [21]:
dfp

,Unnamed: 0,ID_ABONNE,DATE_ACTE_REEL,DATE_DEMARRAGE_PROMO,CPROMO
0,0,51478366401,2021-06-16,2021-06-16,SGEVP01
1,1,52006231501,2021-06-16,2021-06-16,SGEVP01
2,2,51349585101,2021-06-16,2021-06-16,SGEVP01
3,3,52108880501,2021-06-16,2021-06-16,SGEVP01
4,4,51947630401,2021-06-16,2021-06-16,SGEVP01
...,...,...,...,...,...
15733966,15733966,51974732901,2023-05-01,2023-05-01,SGEVP01
15733967,15733967,51923691301,2023-05-01,2023-05-01,SGEVP01
15733968,15733968,50993144501,2023-05-01,2023-05-01,SGEVP01
15733969,15733969,52103629101,2023-05-01,2023-05-01,SGEVP01


•⁠  ⁠transformer la fidélité en un score et essayer de le croiser avec les types de promo
•⁠  ⁠⁠comparer les délais de réabonnements à ce qu’il se fait hors promotion 
•⁠  ⁠⁠on peut dans un premier temps se concentrer sur un seul type d’ODD (ex 15j TC)

créer la table sans ODD

In [50]:
t = file_to_dataframe(path_antoine + 'df_Donnees_Reabos_odd_outer.csv')

In [52]:
t['NB_APPARITIONS'] = t.groupby('ID_ABONNE')['ID_ABONNE'].transform('count')

In [54]:
save_to_csv_file(t, path_antoine + 'df_Donnees_Reabos_odd_outer.csv')

True